In [1]:
import sys, os
p = os.path.abspath('..')
# p = p+r'\config'
sys.path.insert(1, p)

In [2]:
import psycopg2
import pandas as pd
import numpy as np
import config.config as conf
import pickle
import lib.preprocess.preprocess as pp
import lib.preprocess.SectionExtractor as se

In [3]:
htmlp = pp.HTMLParser()
codep = pp.CodeSectionParser()
ts = se.SectionExtractor()

In [4]:
df = pd.read_csv('../../data/part_one_annotation_50_input.csv')

In [5]:
df.columns

Index(['id', 'link', 'Q_difficulty', 'A_difficulty'], dtype='object')

In [6]:
# 'id', 'link', 'Q_Difficulty_class', 'Q_General_rule_set',
#        'Q_Granular_breakdown'

In [7]:
q_df = df[['id', 'link', 'Q_difficulty']]
a_df = df[['id', 'link', 'A_difficulty']]

In [8]:
q_df.head()

,id,link,Q_difficulty
0,77293473,https://stackoverflow.com/questions/77293473,Basic
1,77404924,https://stackoverflow.com/questions/77404924,Basic
2,77249095,https://stackoverflow.com/questions/77249095,Basic
3,77076663,https://stackoverflow.com/questions/77076663,Advanced
4,77019129,https://stackoverflow.com/questions/77019129,Intermediate


In [9]:
q_df

,id,link,Q_difficulty
0,77293473,https://stackoverflow.com/questions/77293473,Basic
1,77404924,https://stackoverflow.com/questions/77404924,Basic
2,77249095,https://stackoverflow.com/questions/77249095,Basic
3,77076663,https://stackoverflow.com/questions/77076663,Advanced
4,77019129,https://stackoverflow.com/questions/77019129,Intermediate
5,76992629,https://stackoverflow.com/questions/76992629,Intermediate
6,77370805,https://stackoverflow.com/questions/77370805,Intermediate
7,76784524,https://stackoverflow.com/questions/76784524,Basic
8,76608783,https://stackoverflow.com/questions/76608783,Intermediate
9,76697269,https://stackoverflow.com/questions/76697269,Intermediate


In [10]:
q_id_str = str([x for x in q_df['id']])[1:-1]
a_id_str = str([x for x in a_df['id']])[1:-1]

In [11]:
q_id_str

'77293473, 77404924, 77249095, 77076663, 77019129, 76992629, 77370805, 76784524, 76608783, 76697269, 76317584, 76570219, 76302571, 76548866, 76541602, 76264395, 76169010, 76169378, 76059076, 76045819, 76085427, 75779691, 75862649, 75637544, 75460677, 75454689, 75283999, 75279786, 75198212, 75057003, 75179892, 75239843, 74778102, 74774389, 74760874, 74751254, 74693871, 74844950, 74804354, 77553477, 77543317, 77511053, 77467788'

In [12]:
q_postypeid="'1'"
a_postypeid="'2'"



In [13]:
# q_sql = """select a.id 
#                 , 'Here is the question about software development. The title of the question is "'||a.title||'". and body of the question is "'||b.body||'". Please answer briefly about the "difficulty class" and "general ruleset" and "grenular breakdown" for the question'
#             from posts a 
#                , postsbody b
#             where a.id = b.id
#               and a.creationdate between '2021-11-30' and '2023-11-30' 
#               and a.posttypeid = {0} 
#               and a.tags like '%<python>%' 
#               and a.id in ({1})
# """

In [14]:
q_sql = """select a.id 
                , a.title
                , b.body
            from posts a 
               , postsbody b
            where a.id = b.id
              and a.creationdate between '2021-11-30' and '2023-11-30' 
              and a.posttypeid = {0} 
              and a.tags like '%<python>%' 
              and a.id in ({1})
"""

In [15]:
conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute(q_sql.format(q_postypeid, q_id_str))
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()

In [16]:
q_output = pd.DataFrame(rows, columns = [
  'id'
  ,'title'
  , 'body'
])


In [17]:
q_output.head()

,id,title,body
0,74693871,Why joblib is not recommended when saving kera...,"<p>According to <a href=""https://keras.io/gett..."
1,74751254,Removing all duplicate images with different f...,<p>I am trying to iterate through a folder and...
2,74760874,How do I extract a binary pattern from a 1D si...,<p>I need help for my school project. I am tak...
3,74774389,How to join these 2 tables by date with ORM,<p>I have two querysets -</p>\n<pre><code>A = ...
4,74778102,Error when trying to save a form in django,<p>I am new to django and I am working on a pr...


In [18]:
q_output

,id,title,body
0,74693871,Why joblib is not recommended when saving kera...,"<p>According to <a href=""https://keras.io/gett..."
1,74751254,Removing all duplicate images with different f...,<p>I am trying to iterate through a folder and...
2,74760874,How do I extract a binary pattern from a 1D si...,<p>I need help for my school project. I am tak...
3,74774389,How to join these 2 tables by date with ORM,<p>I have two querysets -</p>\n<pre><code>A = ...
4,74778102,Error when trying to save a form in django,<p>I am new to django and I am working on a pr...
5,74804354,openCV and pytesseract does not correctly read...,<p>I'm having trouble reading the text correct...
6,74844950,I have a non normalised.db file and I need to ...,<p>I have a non normalised.db file and I need...
7,75057003,How to apply scipy filter in Polars DataFrame,<p>I'm currently using savgol filter in my Dat...
8,75179892,Need to extract values from a dataframe and co...,<p>I have a excel(csv) file and a text file(tx...
9,75198212,Python Asyncio/Await: Having Trouble understan...,<p><strong>Code Block B</strong> is from an As...


In [19]:
q_output['c_body'] = q_output['body'].apply(lambda x : codep(x))

post_str : <p>I am trying to iterate through a folder and delete any file that is a duplicate image (but different name). After running this script all files get deleted except for one. There are at least a dozen unique ones out of about 5,000. Any help understanding why this is happening would be appreciated.</p>
<pre><code>import os
import cv2 

directory = r'C:\Users\Grid\scratch'
 
for filename in os.listdir(directory):
    a=directory+'\\'+filename
    n=(cv2.imread(a))
    q=0
    for filename in os.listdir(directory):
        b=directory+'\\'+filename
        m=(cv2.imread(b))
        comparison = n == m
        equal_arrays = comparison.all()
        if equal_arrays==True and q==1:
            os.remove(b)
        q=1
</code></pre>

post_str : <p>I need help for my school project. I am taking pictures from a webcam and sampling the center of the picture to get a binary reading (0/1).</p>
<p>The signal looks like this:</p>
<p><code>00001111100000000000000001111100000011111000001

In [20]:
q_output['t_body'] = q_output[['c_body', 'body']].apply(lambda row: ts.get_text_section({'body': row['body'], 'code_sections': row['c_body']['code_sections']}), axis=1)

In [21]:
q_output['t_body']

0     <p>According to <a href="https://keras.io/gett...
1     <p>I am trying to iterate through a folder and...
2     <p>I need help for my school project. I am tak...
3     <p>I have two querysets -</p>\n<pre><code>\n\n...
4     <p>I am new to django and I am working on a pr...
5     <p>I'm having trouble reading the text correct...
6     <p>I have  a non normalised.db file and I need...
7     <p>I'm currently using savgol filter in my Dat...
8     <p>I have a excel(csv) file and a text file(tx...
9     <p><strong>Code Block B</strong> is from an As...
10    <p>Im creating dict from database entries:</p>...
11    <p>I'm scraping some information and below is ...
12    <p>I have input dict as below</p>\n<p>sample=<...
13    <p>I'm trying to install <a href="https://pyth...
14    <p>When I try to create a GIF using Pillow one...
15    <p>Is there a method in Python or PyCharm to p...
16    <pre><code>\n\n\n</code></pre>\n<pre><code>\n\...
17    <p>I integrated <code>mypy</code> into my 

In [22]:
q_output['clean_body'] = q_output['t_body'].apply(lambda x : htmlp.get_html_cleaned_str(x))

In [23]:
q_output.head()

,id,title,body,c_body,t_body,clean_body
0,74693871,Why joblib is not recommended when saving kera...,"<p>According to <a href=""https://keras.io/gett...",{'code_sections': []},"<p>According to <a href=""https://keras.io/gett...","According to this keras documentation, pickle ..."
1,74751254,Removing all duplicate images with different f...,<p>I am trying to iterate through a folder and...,"{'code_sections': [{'off_beg': 316, 'off_end':...",<p>I am trying to iterate through a folder and...,I am trying to iterate through a folder and de...
2,74760874,How do I extract a binary pattern from a 1D si...,<p>I need help for my school project. I am tak...,"{'code_sections': [{'off_beg': 666, 'off_end':...",<p>I need help for my school project. I am tak...,I need help for my school project. I am taking...
3,74774389,How to join these 2 tables by date with ORM,<p>I have two querysets -</p>\n<pre><code>A = ...,"{'code_sections': [{'off_beg': 41, 'off_end': ...",<p>I have two querysets -</p>\n<pre><code>\n\n...,I have two querysets -\n\n\n\n\nFirst query gi...
4,74778102,Error when trying to save a form in django,<p>I am new to django and I am working on a pr...,"{'code_sections': [{'off_beg': 370, 'off_end':...",<p>I am new to django and I am working on a pr...,I am new to django and I am working on a proje...


In [24]:
q_output = pd.merge(q_output, q_df[['id', 'Q_difficulty']], on = ['id'] )

In [25]:
q_output['answer'] = 'Difficulty class : '+q_output['Q_difficulty'].map(str)

In [26]:
q_output['question'] = 'Here is the question about software development. The title of the question is <Title>"'+q_output['title'].map(str)+'"</Title>. and body of the question is <Question>"'+q_output['clean_body'].map(str)+'"</Question>. Please answer briefly about the "difficulty" of the stackoverflow question'


In [27]:
q_output[['question', 'answer']]

,question,answer
0,Here is the question about software developmen...,Difficulty class : Advanced
1,Here is the question about software developmen...,Difficulty class : Intermediate
2,Here is the question about software developmen...,Difficulty class : Intermediate
3,Here is the question about software developmen...,Difficulty class : Intermediate
4,Here is the question about software developmen...,Difficulty class : Intermediate
5,Here is the question about software developmen...,Difficulty class : Intermediate
6,Here is the question about software developmen...,Difficulty class : Intermediate
7,Here is the question about software developmen...,Difficulty class : Intermediate
8,Here is the question about software developmen...,Difficulty class : Intermediate
9,Here is the question about software developmen...,Difficulty class : Advanced


In [28]:
q_output[['id', 'question', 'answer']].to_csv('../../data/part_one_q_output.csv')

In [29]:
# [34,  3, 37, 33]
q_output[['body', 'clean_body']].to_csv('chk.csv')

In [30]:
# q_output.iloc[[14, 25, 13, 21], :].to_csv('chk.csv')